**Metrics**

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.spatial.distance import directed_hausdorff

########################### Yo'qotish funksiyalari (Loss Functions) ###########################
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        # Sigmoid orqali chiqishni (inputs) aktivatsiya qiladi
        inputs = torch.sigmoid(inputs)

        # Inputs va targets o'lchamlarini bir tekislikka keltiradi
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # Dice ko'rsatkichi uchun o'zaro kesishgan qismni hisoblaydi
        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        # Dice yo'qotish funksiyasi, uni minimalizatsiya qilish kerak
        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        # Sigmoid orqali chiqishni (inputs) aktivatsiya qiladi
        inputs = torch.sigmoid(inputs)

        # Inputs va targets o'lchamlarini bir tekislikka keltiradi
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # Dice yo'qotish funksiyasini hisoblaydi
        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        # Binary Cross Entropy (BCE) yo'qotish funksiyasi
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')

        # Dice va BCE yo'qotish funksiyalarining yig'indisi
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

###########################  O'lchovlar (Metrics)  ###########################

def precision(y_true, y_pred):
    # Aniqlikni (Precision) hisoblaydi: y_pred bilan y_true o'rtasidagi kesishishni topadi
    intersection = (y_true * y_pred).sum()
    return (intersection + 1e-15) / (y_pred.sum() + 1e-15)

def recall(y_true, y_pred):
    # Eslash (Recall) hisoblaydi: y_true bilan y_pred o'rtasidagi kesishishni topadi
    intersection = (y_true * y_pred).sum()
    return (intersection + 1e-15) / (y_true.sum() + 1e-15)

def F2(y_true, y_pred, beta=2):
    # F2 ko'rsatkichi, precision va recall orqali hisoblanadi
    p = precision(y_true,y_pred)
    r = recall(y_true, y_pred)
    return (1+beta**2.) *(p*r) / float(beta**2*p + r + 1e-15)

def dice_score(y_true, y_pred):
    # Dice ko'rsatkichi hisoblaydi: 2 marta kesishishni (intersection) soniga bo'linadi
    return (2 * (y_true * y_pred).sum() + 1e-15) / (y_true.sum() + y_pred.sum() + 1e-15)

def jac_score(y_true, y_pred):
    # Jaccard indeksi (IoU): y_true va y_pred o'rtasidagi kesishish va yig'ish
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    return (intersection + 1e-15) / (union + 1e-15)

def hd_dist(preds, targets):
    # Hausdorff masofasi: segmentatsiya chegaralari orasidagi maksimal masofa
    haussdorf_dist = directed_hausdorff(preds, targets)[0]
    return haussdorf_dist

**Utils**

In [3]:
import os
import random
import numpy as np
import cv2
from tqdm import tqdm
import torch
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix

#########################  Tasodifiylikni bir xil qilib seeding o'rnatish. #########################

def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

######################### Yangi papka yaratish #########################

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

#########################  Datasetni aralashtirish #########################

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

#########################  Epoch vaqti (bosqichning boshlanish va tugash vaqti orasidagi vaqtni hisoblash). #########################

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#########################  Natijalarni chop etish va saqlash #########################

def print_and_save(file_path, data_str):
    print(data_str)
    with open(file_path, "a") as file:
        file.write(data_str)
        file.write("\n")

#########################  Modellar uchun turli metrikalarni hisoblash #########################

def calculate_metrics(y_true, y_pred):
    ## Tensorlarni numpy formatiga o'tkazish
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    ## Thresholddan keyin taxminlarni (predictions) binar formatga o'tkazish
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)

    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)

    ## Hausdorff masofasi (HD) ni hisoblash
    if len(y_true.shape) == 3:
        score_hd = hd_dist(y_true[0], y_pred[0])
    elif len(y_true.shape) == 4:
        score_hd = hd_dist(y_true[0,0], y_pred[0,0])

    ## Binar formatni bir qatorli qilib joylashtirish (reshape)
    y_pred = y_pred.reshape(-1)
    y_true = y_true.reshape(-1)

    ## Har xil metrikalarni hisoblash
    score_jaccard = jac_score(y_true, y_pred)   # Jaccard indeksi (IoU)
    score_f1 = dice_score(y_true, y_pred)       # Dice/F1 score
    score_recall = recall(y_true, y_pred)       # Recall
    score_precision = precision(y_true, y_pred) # Precision
    score_fbeta = F2(y_true, y_pred)            # F2 score (ko'proq recall'ga urg'u beradi)
    score_acc = accuracy_score(y_true, y_pred)  # Aniqlik (Accuracy)

    ## Metrikalar ro'yxatini qaytarish
    return [score_jaccard, score_f1, score_recall, score_precision, score_acc, score_fbeta, score_hd]

In [4]:
!pip install ptflops

**Model**

In [5]:
import torch
import torch.nn as nn

# Squeeze-and-Excitation bloki
class Squeeze_Excitation(nn.Module):
    def __init__(self, channel, r=8):
        super().__init__()

        # Adaptive o'rtacha hovuzlash
        self.pool = nn.AdaptiveAvgPool2d(1)
        # Neural tarmoq qismi
        self.net = nn.Sequential(
            nn.Linear(channel, channel // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // r, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, inputs):
        b, c, _, _ = inputs.shape
        x = self.pool(inputs).view(b, c)
        x = self.net(x).view(b, c, 1, 1)
        x = inputs * x
        return x

# Stem blok (asosiy kirish bloki)
class Stem_Block(nn.Module):
    def __init__(self, in_c, out_c, stride):
        super().__init__()

        # Birinchi konvolyutsiya bloki
        self.c1 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
        )

        # Shortcut (o'tish yo'li) konvolyutsiya
        self.c2 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=1, stride=stride, padding=0),
            nn.BatchNorm2d(out_c),
        )

        # Squeeze-Excitation bloki yordamida e'tibor mexanizmi
        self.attn = Squeeze_Excitation(out_c)

    def forward(self, inputs):
        x = self.c1(inputs)
        s = self.c2(inputs)
        y = self.attn(x + s) # Shortcut va asosiy yo'ldan kelgan ma'lumotlarni yig'ish
        return y

# ResNet bloki (asosiy blok)
class ResNet_Block(nn.Module):
    def __init__(self, in_c, out_c, stride):
        super().__init__()

        # Konvolyutsiya va batch normalization
        self.c1 = nn.Sequential(
            nn.BatchNorm2d(in_c),
            nn.ReLU(),
            nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, stride=stride),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1)
        )

        # Shortcut (o'tish yo'li)
        self.c2 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=1, stride=stride, padding=0),
            nn.BatchNorm2d(out_c),
        )

        # E'tibor mexanizmi
        self.attn = Squeeze_Excitation(out_c)

    def forward(self, inputs):
        x = self.c1(inputs)
        s = self.c2(inputs)
        y = self.attn(x + s) # Shortcut bilan ma'lumotlarni birlashtirish
        return y

# ASPP bloki (Atrous Spatial Pyramid Pooling)
class ASPP(nn.Module):
    def __init__(self, in_c, out_c, rate=[1, 6, 12, 18]):
        super().__init__()

        # Kengaytirilgan konvolyutsiyalar (dilation)
        self.c1 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, dilation=rate[0], padding=rate[0]),
            nn.BatchNorm2d(out_c)
        )

        self.c2 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, dilation=rate[1], padding=rate[1]),
            nn.BatchNorm2d(out_c)
        )

        self.c3 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, dilation=rate[2], padding=rate[2]),
            nn.BatchNorm2d(out_c)
        )

        self.c4 = nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, dilation=rate[3], padding=rate[3]),
            nn.BatchNorm2d(out_c)
        )

        self.c5 = nn.Conv2d(out_c, out_c, kernel_size=1, padding=0)


    def forward(self, inputs):
        # Har xil darajadagi kengaytirilgan konvolyutsiya natijalari
        x1 = self.c1(inputs)
        x2 = self.c2(inputs)
        x3 = self.c3(inputs)
        x4 = self.c4(inputs)
        x = x1 + x2 + x3 + x4
        y = self.c5(x) # Yig'ilgan natijalarni bitta qatlam orqali o'tkazish
        return y

# Diqqat bloki (Attention Block)
class Attention_Block(nn.Module):
    def __init__(self, in_c):
        super().__init__()
        out_c = in_c[1]

        # G ma'lumot uchun konvolyutsiya
        self.g_conv = nn.Sequential(
            nn.BatchNorm2d(in_c[0]),
            nn.ReLU(),
            nn.Conv2d(in_c[0], out_c, kernel_size=3, padding=1),
            nn.MaxPool2d((2, 2))
        )

        # X ma'lumot uchun konvolyutsiya
        self.x_conv = nn.Sequential(
            nn.BatchNorm2d(in_c[1]),
            nn.ReLU(),
            nn.Conv2d(in_c[1], out_c, kernel_size=3, padding=1),
        )

        # G va X ma'lumotlarni birlashtirish
        self.gc_conv = nn.Sequential(
            nn.BatchNorm2d(in_c[1]),
            nn.ReLU(),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
        )

    def forward(self, g, x):
        g_pool = self.g_conv(g)
        x_conv = self.x_conv(x)
        gc_sum = g_pool + x_conv
        gc_conv = self.gc_conv(gc_sum)
        y = gc_conv * x
        return y

# Dekoder bloki (Decoder Block)
class Decoder_Block(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()

        # E'tibor bloki bilan dekodlash
        self.a1 = Attention_Block(in_c)
        self.up = nn.Upsample(scale_factor=2, mode="nearest") # Upsampling
        self.r1 = ResNet_Block(in_c[0]+in_c[1], out_c, stride=1) # ResNet bloki

    def forward(self, g, x):
        d = self.a1(g, x)
        d = self.up(d)
        d = torch.cat([d, g], axis=1) # Upsample qilingan ma'lumotlarni birlashtirish
        d = self.r1(d)
        return d

# ResUNet++ ni qurish
class build_resunetplusplus(nn.Module):
    def __init__(self):
        super().__init__()

        # Kodlash bosqichlari
        self.c1 = Stem_Block(3, 16, stride=1)
        self.c2 = ResNet_Block(16, 32, stride=2)
        self.c3 = ResNet_Block(32, 64, stride=2)
        self.c4 = ResNet_Block(64, 128, stride=2)

        # ASPP
        self.b1 = ASPP(128, 256)

        # Dekodlash bosqichlari
        self.d1 = Decoder_Block([64, 256], 128)
        self.d2 = Decoder_Block([32, 128], 64)
        self.d3 = Decoder_Block([16, 64], 32)

        # Yakuniy ASPP va chiqish konvolyutsiyasi
        self.aspp = ASPP(32, 16)
        self.output = nn.Conv2d(16, 1, kernel_size=1, padding=0)

    def forward(self, inputs):
        # Kodlash bosqichlari
        c1 = self.c1(inputs)
        c2 = self.c2(c1)
        c3 = self.c3(c2)
        c4 = self.c4(c3)

        # ASPP bloki
        b1 = self.b1(c4)

        # Dekodlash bosqichlari
        d1 = self.d1(c3, b1)
        d2 = self.d2(c2, d1)
        d3 = self.d3(c1, d2)

        # Yakuniy chiqish
        output = self.aspp(d3)
        output = self.output(output)

        return output


if __name__ == "__main__":
    model = build_resunetplusplus()

    # Model murakkabligi va parametrlari
    from ptflops import get_model_complexity_info
    flops, params = get_model_complexity_info(model, input_res=(3, 256, 256), as_strings=True, print_per_layer_stat=False)
    print('      - Flops:  ' + flops)
    print('      - Params: ' + params)

      - Flops:  15.85 GMac
      - Params: 4.06 M


**Train**

In [7]:
import os
import random
import time
import datetime
import numpy as np
import albumentations as A
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
from tqdm import tqdm

# Tasvirlar, maskalar va bashoratlarni saqlash uchun funksiya
def save_images(images, masks, preds, epoch, output_dir="results_images"):
    os.makedirs(output_dir, exist_ok=True)

    for i in range(len(images)):
        image = images[i].cpu().numpy().transpose(1, 2, 0) * 255.0
        mask = masks[i].cpu().numpy().transpose(1, 2, 0) * 255.0
        pred = preds[i].cpu().numpy().transpose(1, 2, 0) * 255.0

        image = Image.fromarray(image.astype(np.uint8))
        mask = Image.fromarray(mask.astype(np.uint8))
        pred = Image.fromarray(pred.astype(np.uint8))

        image.save(os.path.join(output_dir, f"epoch_{epoch}_image_{i}.png"))
        mask.save(os.path.join(output_dir, f"epoch_{epoch}_mask_{i}.png"))
        pred.save(os.path.join(output_dir, f"epoch_{epoch}_pred_{i}.png"))

# Ma'lumotlarni yuklash uchun funksiya
def load_data(path):
    images_path = sorted(glob(os.path.join(path, "images", "*.jpg")))
    masks_path = sorted(glob(os.path.join(path, "masks", "*.png")))

    # Ma'lumotlarni o'quv, validatsiya va sinov to'plamlariga ajratish
    train_x, temp_x, train_y, temp_y = train_test_split(images_path, masks_path, test_size=0.4, random_state=42)
    valid_x, test_x, valid_y, test_y = train_test_split(temp_x, temp_y, test_size=0.5, random_state=42)

    return [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]

# Ma'lumotlar to'plamini yaratish uchun sinf
class DATASET(Dataset):
    def __init__(self, images_path, masks_path, size, transform=None):
        super().__init__()

        self.images_path = images_path
        self.masks_path = masks_path
        self.size = size
        self.transform = transform
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        # Tasvir va maskalarni o'qish
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        image = cv2.resize(image, self.size)
        image = np.transpose(image, (2, 0, 1))
        image = image / 255.0

        mask = cv2.resize(mask, self.size)
        mask = np.expand_dims(mask, axis=0)
        mask = mask / 255.0

        return image, mask

    def __len__(self):
        return self.n_samples

# Modelni o'qitish uchun funksiya
def train(model, loader, optimizer, loss_fn, device):
    model.train()
    epoch_loss = 0.0
    epoch_jac = 0.0
    epoch_f1 = 0.0
    epoch_recall = 0.0
    epoch_precision = 0.0

    for i, (x, y) in enumerate(tqdm(loader)):
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        y_pred = torch.sigmoid(y_pred)
        batch_jac = []
        batch_f1 = []
        batch_recall = []
        batch_precision = []

        for yt, yp in zip(y, y_pred):
            score = calculate_metrics(yt, yp)
            batch_jac.append(score[0])
            batch_f1.append(score[1])
            batch_recall.append(score[2])
            batch_precision.append(score[3])

        epoch_jac += np.mean(batch_jac)
        epoch_f1 += np.mean(batch_f1)
        epoch_recall += np.mean(batch_recall)
        epoch_precision += np.mean(batch_precision)

    epoch_loss /= len(loader)
    epoch_jac /= len(loader)
    epoch_f1 /= len(loader)
    epoch_recall /= len(loader)
    epoch_precision /= len(loader)

    return epoch_loss, [epoch_jac, epoch_f1, epoch_recall, epoch_precision]

# Modelni baholash uchun funksiya
def evaluate(model, loader, loss_fn, device):
    model.eval()
    epoch_loss = 0.0
    epoch_jac = 0.0
    epoch_f1 = 0.0
    epoch_recall = 0.0
    epoch_precision = 0.0

    with torch.no_grad():
        for i, (x, y) in enumerate(tqdm(loader)):
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

            y_pred = torch.sigmoid(y_pred)
            batch_jac = []
            batch_f1 = []
            batch_recall = []
            batch_precision = []

            for yt, yp in zip(y, y_pred):
                score = calculate_metrics(yt, yp)
                batch_jac.append(score[0])
                batch_f1.append(score[1])
                batch_recall.append(score[2])
                batch_precision.append(score[3])

            epoch_jac += np.mean(batch_jac)
            epoch_f1 += np.mean(batch_f1)
            epoch_recall += np.mean(batch_recall)
            epoch_precision += np.mean(batch_precision)

        epoch_loss /= len(loader)
        epoch_jac /= len(loader)
        epoch_f1 /= len(loader)
        epoch_recall /= len(loader)
        epoch_precision /= len(loader)

        # Tasodifiy rasmni tanlash va uning natijalarini olish
        rand_index = random.randint(0, len(loader.dataset) - 1)
        sample_img, sample_mask = loader.dataset[rand_index]
        sample_img = torch.tensor(sample_img, dtype=torch.float32).unsqueeze(0).to(device)
        sample_pred = model(sample_img)
        sample_pred = torch.sigmoid(sample_pred).detach().cpu().squeeze().numpy()

    return epoch_loss, [epoch_jac, epoch_f1, epoch_recall, epoch_precision], sample_img.cpu().squeeze().numpy(), sample_mask, sample_pred

if __name__ == "__main__":
    seeding(42)  # Tasodifiylikni boshqarish uchun urug'ni o'rnatish
    create_dir("files")  # Fayllar saqlanadigan papkani yaratish

    train_log_path = "files/train_log.txt"
    if os.path.exists(train_log_path):
        print("Log fayli mavjud")
    else:
        train_log = open(train_log_path, "w")
        train_log.write("\n")
        train_log.close()

    datetime_object = str(datetime.datetime.now())
    print_and_save(train_log_path, datetime_object)  # Logga hozirgi sana va vaqtni yozish
    print("")

    # Hyperparametrlarni belgilash
    image_size = 256
    size = (image_size, image_size)
    batch_size = 16
    num_epochs = 70
    lr = 0.0001
    early_stopping_patience = 50
    checkpoint_path = "files/checkpoint.pth"
    path = "/kaggle/input/newdataset1708/dataset"

    data_str = f"Image Size: {size}\nBatch Size: {batch_size}\nLR: {lr}\nEpochs: {num_epochs}\n"
    data_str += f"Early Stopping Patience: {early_stopping_patience}\n"
    print_and_save(train_log_path, data_str)

    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
    train_x, train_y = shuffling(train_x, train_y)  # Ma'lumotlarni aralashtirish

    data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}\n"
    print_and_save(train_log_path, data_str)

    # Tasvirlarni ko'paytirish uchun transformatsiyalar
    transform = A.Compose([
        A.Rotate(limit=35, p=0.3),
        A.HorizontalFlip(p=0.3),
        A.VerticalFlip(p=0.3),
        A.CoarseDropout(p=0.3, max_holes=10, max_height=32, max_width=32)
    ])

    train_dataset = DATASET(train_x, train_y, size, transform=transform)
    valid_dataset = DATASET(valid_x, valid_y, size, transform=None)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

    valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    device = torch.device('cuda')  # CUDA qurilmasiga o'tish
    model = build_resunetplusplus()  # Modelni yaratish
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    loss_fn = DiceBCELoss()  # Yo'qotish funktsiyasini tanlash
    loss_name = "BCE Dice Loss"
    data_str = f"Optimizer: Adam\nLoss: {loss_name}\n"
    print_and_save(train_log_path, data_str)

    best_valid_metrics = 0.0
    early_stopping_count = 0

    for epoch in range(num_epochs):
        start_time = time.time()

        # O'qitish
        train_loss, train_metrics = train(model, train_loader, optimizer, loss_fn, device)
        valid_loss, valid_metrics, sample_img, sample_mask, sample_pred = evaluate(model, valid_loader, loss_fn, device)
        scheduler.step(valid_loss)

        # Eng yaxshi modelni saqlash
        if valid_metrics[1] > best_valid_metrics:
            data_str = f"Valid F1 yaxshilandi: {best_valid_metrics:2.4f} dan {valid_metrics[1]:2.4f} ga. Tekshirishni saqlamoqda: {checkpoint_path}"
            print_and_save(train_log_path, data_str)

            best_valid_metrics = valid_metrics[1]
            torch.save(model.state_dict(), checkpoint_path)
            early_stopping_count = 0

        elif valid_metrics[1] < best_valid_metrics:
            early_stopping_count += 1

        # Namuna rasm, maskalar va bashoratlarni saqlash
        sample_img = np.transpose(sample_img, (1, 2, 0))  # (H, W, C) formatiga o'tkazish
        if sample_mask.ndim == 3:
            sample_mask = np.squeeze(sample_mask, axis=0)  # (H, W) formatiga o'tkazish
        if sample_pred.ndim == 3:
            sample_pred = np.squeeze(sample_pred, axis=0)  # (H, W) formatiga o'tkazish

        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].imshow(sample_img)
        ax[0].set_title('Original Image')
        ax[1].imshow(sample_mask, cmap='gray')
        ax[1].set_title('Ground Truth Mask')
        ax[2].imshow(sample_pred, cmap='gray')
        ax[2].set_title('Predicted Mask')
        plt.savefig(f'files/epoch_{epoch+1}_sample.png')
        plt.close(fig)

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        data_str = f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n"
        data_str += f"\tTrain Loss: {train_loss:.4f} - Jaccard: {train_metrics[0]:.4f} - F1: {train_metrics[1]:.4f} - Recall: {train_metrics[2]:.4f} - Precision: {train_metrics[3]:.4f}\n"
        data_str += f"\t Val. Loss: {valid_loss:.4f} - Jaccard: {valid_metrics[0]:.4f} - F1: {valid_metrics[1]:.4f} - Recall: {valid_metrics[2]:.4f} - Precision: {valid_metrics[3]:.4f}\n"
        print(f"Epoch time: {epoch_mins} min, {epoch_secs} s")
        print_and_save(train_log_path, data_str)

        # Erta to'xtatish shartlarini tekshirish
        if early_stopping_count == early_stopping_patience:
            data_str = f"Erta to'xtatish: validatsiya yo'qotishi {early_stopping_patience} ta ketma-ketlikdan keyin yaxshilanmadi.\n"
            print_and_save(train_log_path, data_str)
            break

Log fayli mavjud
2024-09-02 10:19:40.616978

Image Size: (256, 256)
Batch Size: 16
LR: 0.0001
Epochs: 70
Early Stopping Patience: 50

Dataset Size:
Train: 4503 - Valid: 1501 - Test: 1502

Optimizer: Adam
Loss: BCE Dice Loss



100%|██████████| 94/94 [00:44<00:00,  2.09it/s]


Valid F1 yaxshilandi: 0.0000 dan 0.8564 ga. Tekshirishni saqlamoqda: files/checkpoint.pth
Epoch time: 6 min, 37 s
Epoch: 01 | Epoch Time: 6m 37s
	Train Loss: 0.7908 - Jaccard: 0.6856 - F1: 0.7973 - Recall: 0.8296 - Precision: 0.7997
	 Val. Loss: 0.5901 - Jaccard: 0.7653 - F1: 0.8564 - Recall: 0.8842 - Precision: 0.8514



100%|██████████| 94/94 [00:44<00:00,  2.11it/s]


Valid F1 yaxshilandi: 0.8564 dan 0.8838 ga. Tekshirishni saqlamoqda: files/checkpoint.pth
Epoch time: 6 min, 33 s
Epoch: 02 | Epoch Time: 6m 33s
	Train Loss: 0.5935 - Jaccard: 0.7662 - F1: 0.8572 - Recall: 0.8862 - Precision: 0.8504
	 Val. Loss: 0.4921 - Jaccard: 0.8056 - F1: 0.8838 - Recall: 0.9092 - Precision: 0.8733



...

Valid F1 yaxshilandi: 0.9410 dan 0.9410 ga. Tekshirishni saqlamoqda: files/checkpoint.pth
Epoch time: 6 min, 37 s
Epoch: 68 | Epoch Time: 6m 37s
	Train Loss: 0.2219 - Jaccard: 0.9038 - F1: 0.9473 - Recall: 0.9555 - Precision: 0.9418
	 Val. Loss: 0.2575 - Jaccard: 0.8950 - F1: 0.9410 - Recall: 0.9529 - Precision: 0.9344



100%|██████████| 94/94 [00:45<00:00,  2.09it/s]


Epoch time: 6 min, 36 s
Epoch: 69 | Epoch Time: 6m 36s
	Train Loss: 0.2223 - Jaccard: 0.9036 - F1: 0.9471 - Recall: 0.9553 - Precision: 0.9416
	 Val. Loss: 0.2575 - Jaccard: 0.8951 - F1: 0.9410 - Recall: 0.9551 - Precision: 0.9321



100%|██████████| 94/94 [00:45<00:00,  2.09it/s]


Valid F1 yaxshilandi: 0.9410 dan 0.9412 ga. Tekshirishni saqlamoqda: files/checkpoint.pth
Epoch time: 6 min, 36 s
Epoch: 70 | Epoch Time: 6m 36s
	Train Loss: 0.2229 - Jaccard: 0.9033 - F1: 0.9470 - Recall: 0.9551 - Precision: 0.9416
	 Val. Loss: 0.2557 - Jaccard: 0.8953 - F1: 0.9412 - Recall: 0.9527 - Precision: 0.9348



In [8]:
import shutil

# Zip qilish uchun folder yo'li va zip fayl nomi
folder_path = "/kaggle/working/files"  # Zip qilish kerak bo'lgan folder
zip_file_name = "/kaggle/working/files.zip"  # Zip faylning saqlanish yo'li va nomi

# Folderni zip qilish
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', folder_path)

print(f"{zip_file_name} muvaffaqiyatli saqlandi.")


/kaggle/working/files.zip muvaffaqiyatli saqlandi.


**Test**

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
import os

def load_model(checkpoint_path, model, device):
    model.load_state_dict(torch.load(checkpoint_path))  # Modelni saqlangan checkpoint'dan yuklaydi
    model = model.to(device)  # Modelni tanlangan qurilmaga (CPU yoki GPU) o'tkazadi
    model.eval()  # Modelni baholash rejimiga o'tkazadi
    return model


def visualize_results(images, masks, preds, idx=0):
    image = images[idx]  # Indeks bo'yicha rasmni tanlaydi
    mask = masks[idx]  # Indeks bo'yicha haqiqiy maskani tanlaydi
    pred = preds[idx]  # Indeks bo'yicha bashorat qilingan maskani tanlaydi

    image = np.transpose(image, (1, 2, 0))  # Rasmni (C, H, W) formatidan (H, W, C) formatiga o'tkazadi
    if mask.ndim == 3:
        mask = np.squeeze(mask, axis=0)  # Agar maskaning o'lchami 3D bo'lsa, uni 2D ga o'tkazadi
    if pred.ndim == 3:
        pred = np.squeeze(pred, axis=0)  # Agar bashorat qilingan maskaning o'lchami 3D bo'lsa, uni 2D ga o'tkazadi

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))  # Uchta subplots yaratadi
    ax[0].imshow(image)  # Original rasmini ko'rsatadi
    ax[0].set_title('Original Image')  # Title qo'shadi
    ax[1].imshow(mask, cmap='gray')  # Haqiqiy maskani ko'rsatadi, rang sxemasi qora va oq
    ax[1].set_title('Ground Truth Mask')  # Title qo'shadi
    ax[2].imshow(pred, cmap='gray')  # Bashorat qilingan maskani ko'rsatadi, rang sxemasi qora va oq
    ax[2].set_title('Predicted Mask')  # Title qo'shadi
    plt.show()  # Grafikni ko'rsatadi


def test_model(model, loader, loss_fn, device):
    model.eval()  # Modelni baholash rejimiga o'tkazadi
    test_loss = 0.0
    test_jac = 0.0
    test_f1 = 0.0
    test_recall = 0.0
    test_precision = 0.0

    images, masks, preds = [], [], []
    with torch.no_grad():  # Gradient hisoblashni o'chiradi
        for i, (x, y) in enumerate(loader):
            x = x.to(device, dtype=torch.float32)  # Kirish ma'lumotlarini tanlangan qurilmaga o'tkazadi
            y = y.to(device, dtype=torch.float32)  # Haqiqiy maskani tanlangan qurilmaga o'tkazadi

            y_pred = model(x)  # Modeldan bashorat qilingan maskani olish
            loss = loss_fn(y_pred, y)  # Yo'qotishni hisoblash
            test_loss += loss.item()  # Yo'qotishni qo'shadi

            y_pred = torch.sigmoid(y_pred)  # Sigmoid funksiyasini qo'llaydi

            x_cpu = x.cpu().numpy()  # Kirish ma'lumotlarini CPU ga ko'chiradi va numpy massivga aylantiradi
            y_cpu = y.cpu().numpy()  # Haqiqiy maskani CPU ga ko'chiradi va numpy massivga aylantiradi
            y_pred_cpu = y_pred.cpu().numpy()  # Bashorat qilingan maskani CPU ga ko'chiradi va numpy massivga aylantiradi

            images.extend(x_cpu)  # Kirish rasmlarini ro'yxatga qo'shadi
            masks.extend(y_cpu)  # Haqiqiy maskalarni ro'yxatga qo'shadi
            preds.extend(y_pred_cpu)  # Bashorat qilingan maskalarni ro'yxatga qo'shadi

            batch_jac, batch_f1, batch_recall, batch_precision = [], [], [], []
            for yt, yp in zip(y, y_pred):
                score = calculate_metrics(yt, yp)  # Metrikalarni hisoblaydi
                batch_jac.append(score[0])
                batch_f1.append(score[1])
                batch_recall.append(score[2])
                batch_precision.append(score[3])

            test_jac += np.mean(batch_jac)  # Jaccard ko'rsatkichini qo'shadi
            test_f1 += np.mean(batch_f1)  # F1 ko'rsatkichini qo'shadi
            test_recall += np.mean(batch_recall)  # Recall ko'rsatkichini qo'shadi
            test_precision += np.mean(batch_precision)  # Precision ko'rsatkichini qo'shadi

    test_loss /= len(loader)  # O'rtacha yo'qotishni hisoblaydi
    test_jac /= len(loader)  # O'rtacha Jaccard ko'rsatkichini hisoblaydi
    test_f1 /= len(loader)  # O'rtacha F1 ko'rsatkichini hisoblaydi
    test_recall /= len(loader)  # O'rtacha Recall ko'rsatkichini hisoblaydi
    test_precision /= len(loader)  # O'rtacha Precision ko'rsatkichini hisoblaydi

    visualize_results(images, masks, preds)  # Natijalarni vizualizatsiya qiladi

    print(f"Test Loss: {test_loss:.4f}")  # Test yo'qotishni chiqaradi
    print(f"Test Metrics:\nJaccard: {test_jac:.4f} - F1: {test_f1:.4f} - Recall: {test_recall:.4f} - Precision: {test_precision:.4f}")  # Test ko'rsatkichlarini chiqaradi


if __name__ == "__main__":
    seeding(42)  # Tasodifiy sonlar uchun urug'larni o'rnatadi

    checkpoint_path = "/kaggle/working/files/checkpoint.pth"  # Modelni yuklash uchun checkpoint yo'li
    path = "/kaggle/input/newdataset1708/dataset"  # Ma'lumotlarni yuklash uchun yo'l
    batch_size = 16  # Batch hajmi
    image_size = 256  # Rasm o'lchami
    size = (image_size, image_size)  # Rasm o'lchamining tuple'i

    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)  # Ma'lumotlarni yuklaydi

    test_dataset = DATASET(test_x, test_y, size)  # Test datasetini yaratadi
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )  # Test dataloader'ini yaratadi

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU yoki CPU tanlaydi
    model = build_resunetplusplus()  # Modelni quradi
    model = load_model(checkpoint_path, model, device)  # Modelni yuklaydi
    loss_fn = DiceBCELoss()  # Yo'qotish funksiyasini yaratadi
    test_model(model, test_loader, loss_fn, device)  # Modelni sinaydi

**Draw**

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as T
from model import build_resunetplusplus

def load_model():
    model = build_resunetplusplus()
    state_dict = torch.load("checkpoint.pth", map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Tasvirni preprocessing qilish
def preprocess(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = T.Compose([
        T.Resize((512, 512)),  # Tasvirni model kirish o'lchamiga moslashtirish
        T.ToTensor(),          # Tasvirni tensor shakliga o'tkazish
    ])
    return transform(image).unsqueeze(0), image  # Batch o'lchamini qo'shish va asl rasmni qaytarish

# Maskada chegara chiziqlarini chizish
def draw_boundaries_on_image(original_image, mask):
    # Predicted maskni ikkilik formatga o'tkazish (thresholding)
    binary_mask = (mask > 0.5).astype(np.uint8)  # 0.5 qiymatidan yuqori bo'lgan ostonaviy

    # Ikkilik maskni original tasvir o'lchamiga moslashtirish
    binary_mask_resized = cv2.resize(binary_mask, (original_image.width, original_image.height), interpolation=cv2.INTER_NEAREST)

    # Ikkilik maskada konturlarni topish
    contours, _ = cv2.findContours(binary_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Original tasvirni numpy arrayga o'tkazish
    image_with_boundaries = np.array(original_image)

    # Chegaralarni tasvirga chizish
    for contour in contours:
        cv2.drawContours(image_with_boundaries, [contour], -1, (0, 0, 255), 2)  # (0, 0, 255) - qizil rang

    return image_with_boundaries

# Modelni yuklab olish va tasvirni preprocess qilish
image_path = []
for i in range(len(image_path)):
    input_image, original_image = preprocess(image_path[i])

    # Model orqali taxmin qilish
    with torch.no_grad():
        model = load_model()
        output = model(input_image)
        predicted_mask = torch.sigmoid(output).squeeze().cpu().numpy()

    # Tasvirda chegara chizish
    image_with_boundaries = draw_boundaries_on_image(original_image, predicted_mask)

    # Natijalarni ko'rsatish
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title('Kirish Tasvir')
    plt.imshow(original_image)

    plt.subplot(1, 3, 2)
    plt.title('Taxmin Qilingan Mask')
    plt.imshow(predicted_mask, cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Chegaralari Chizilgan Tasvir')
    plt.imshow(image_with_boundaries)
    plt.savefig(f"{i}_results.png")